# 4.6 Validate LSTM Model

In this notebook we will validate the results of the accurate LSTM model trained in section 4.4. We will also investigate modifying the model components so that they are viable as a life trading model. 

Currently, the model simply predicts up/down price movement in the next bar. While it does so exceedingly accurrately, constraints include trading on minimally moving bars and bid/ask spreads. Because of the existance of the spread, if a bar moves less than the spread it is possible to both be correct about the direction and lose money on trading it. In order to ensure that the model generates return, we need to be weary of trading when price movement falls below the bid/ask spread. 

#### Steps

I) Confirm the model works in a lossless environment (w/o bid/ask spread)  
II) Modify the model to protect against minimal movement environments (w/ bid/ask spread)  

### I) Confirm the model works in a lossless environment (w/o bid/ask spread)  

In order to confirm the model behaves as expected, we will take the following steps: 

1) Investigate actual predictions from the model against validation data  
2) Using validation data, model returns by backtesting the system  
3) Modify the backtesting system to include some penalty for trading (transaction costs)  

#### 1) Investigate actual predictions from the model against validation data

i) Load a dataset (a single stock)  
ii) Split for the most recent data available, validation data that wasn't included in test to begin with  
iii) predict on the data, look at accuracy and heatmap of predictions  
iv) Plot the data with predictions  
v) Backtest the results of trading   

In [1]:
import pandas as pd
import numpy as np
from extract import load_set
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pd.set_option('display.max_columns', 130)
pd.set_option('display.max_rows', 100)

In [2]:
data_dir = './data/prepared/august25screenfixed/'
suffix = ''

stocks = [f.split('.')[0] for f in listdir(data_dir) if isfile(join(data_dir, f))]
stocks[0]

'RRR'

In [200]:
stocks

['RRR',
 'RLGY',
 'HOME',
 'FDX',
 'MIK',
 'WSM',
 'NVDA',
 'DE',
 'TGT',
 'LULU',
 'FBHS',
 'XRX',
 'CX',
 'EAT',
 'DHI',
 'ICE',
 'EXPI',
 'CLNY',
 'IMVT',
 'ELAN',
 'PACB',
 'PENN',
 'REAL',
 'NKE',
 'BBY',
 'HTHT',
 'GRPN',
 'CZR',
 'GDDY',
 'LOW']

In [3]:
df = load_set(stocks[0], data_dir, suffix)
df.head()

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_maxcov,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,close_tan,close_tan2,close_xsinx,close_xcosx,close_sinxcosx,close_xsinxcosx,close_xsinx2,close_xcosx2,close_sinxcosx2,close_xsinxcosx2,close_xtanx,close_xtanx2,decision,D2
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,-0.001156,0.490591,-0.002573,2.833824,-611.936535,-0.000390,0.451017,2.217211,1.726659e+05,0.001653,24453.239631,5.365601,5.816618,0.000767,0.893118,0.607740,-1.306702,-0.503655,-0.000002,-0.000703,0.001004,-0.000001,-8.865898e-07,-0.001033,0.000685,-7.918291e-07,0.001511,0.000582,0.0,0.0
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,2.322576e+05,-0.000633,8844,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,1.819360e+05,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-9.187537e-06,0.000012,0.006850,0.001223,-9.820883e-06,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,2.107880e+05,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,1.408935e+05,-0.001990,24450.356426,2.482395,2.934083,-0.000885,-0.978547,-0.790484,-0.774819,-0.210541,0.000016,0.006346,0.001573,-0.000013,7.103888e-06,0.007855,0.000866,-6.951488e-06,0.006220,0.001690,0.0,0.0
8843,17.35,17.44,17.35,17.44,25,2020-08-25 22:36:00,2020-08-25,22,36,1356,RRR,17.35,0.09,0.005187,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0

In [4]:
#df.loc[6064:6060]

In [5]:
#df[df.isna().sum(axis=1) > 0]

In [6]:
df['target'] = df['%close'].shift(1)

In [7]:
df[['close','%close','target']]

,close,%close,target
8845,17.28,-0.001156,NaN
8844,17.30,-0.008028,-0.001156
8843,17.44,0.005187,-0.008028
8842,17.35,0.001732,0.005187
8841,17.32,0.000000,0.001732
...,...,...,...
4,10.72,0.006573,-0.008396
3,10.65,0.000000,0.006573
2,10.65,-0.004673,0.000000
1,10.70,-0.027273,-0.004673


In [8]:
df['open'][::-1].rolling(30).mean()[::-1]

8845    17.359000
8844    17.362333
8843    17.364333
8842    17.362667
8841    17.367000
          ...    
4             NaN
3             NaN
2             NaN
1             NaN
0             NaN
Name: open, Length: 8846, dtype: float64

In [9]:
e = df.shape[0]
l = e - 4000 - 59

e, l

(8846, 4787)

The train/test data 

In [10]:
df.iloc[:l]['target']

8845         NaN
8844   -0.001156
8843   -0.008028
8842    0.005187
8841    0.001732
          ...   
4063    0.002493
4062   -0.005666
4061    0.002841
4060    0.000711
4059    0.003566
Name: target, Length: 4787, dtype: float64

In [107]:
df = load_set(stocks[0], data_dir, suffix)

for col in ['open','high','low','close']:
        df[col] = df[col] - df[col][::-1].rolling(30).mean()[::-1]

df = df.dropna(axis=0)

e = df.shape[0]
l = e - 4000 - 59

df['target'] = df['%close'].shift(1)

y_ = df.iloc[1:4001]['target']

In [497]:
X = np.zeros([4000,116,60])

df = load_set(stocks[0], data_dir, suffix)


for col in ['open','high','low','close']:
        df[col] = df[col] - df[col][::-1].rolling(30).mean()[::-1]

df = df.dropna(axis=0)
        
e = df.shape[0]
l = e - 4000 - 59


df['target'] = df['%close'].shift(1)

y_ = df.iloc[l:e-59]['target']
#y_ = df.iloc[1:4001]['target']
y = y_.to_numpy() 

drop_cols = ['datetime',
            'date',
            'min_num',
            'SYMBOL',
            'prev_close',
            'diff_1',
            'time',
            'decision',
            'open_maxcov',
            'high_maxcov',
            'low_maxcov',
            'close_maxcov',
            'pct_change',
            'D2',
            'target']

df.drop(drop_cols, axis=1, inplace=True)

# mx = MinMaxScaler()
# mx = mx.fit(df)
# scaled_features = mx.transform(df)
# df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

# dt = df.transpose()

for j, i in enumerate(range(1, 4001)):
     X[j] = dt.iloc[:, i:i+60].to_numpy()
    
y.shape, X.shape
l

4756

In [498]:
y = np.zeros([1,4000])

#for k, stock in enumerate(stocks):
df = load_set(stocks[0], data_dir, suffix)

df = df.dropna(axis=0)

e = df.shape[0]
l = e - 4000 - 59

df['y'] = df['%close'].shift(1)
y_ = df.iloc[1+4:4001+4]['y']
y[0] = y_.to_numpy() 
l

4760

In [513]:
df.iloc[1]

open                  17.3
high                  17.3
low                   17.3
close                 17.3
volume                 200
                   ...    
close_xtanx     0.00621988
close_xtanx2    0.00169012
decision                 0
D2                       0
y              -0.00115607
Name: 8844, Length: 131, dtype: object

In [512]:
df.iloc[5]

open                17.32
high                17.32
low                 17.32
close               17.32
volume                  1
                  ...    
close_xtanx     0.0297185
close_xtanx2    0.0751944
decision                0
D2                      0
y                       0
Name: 8840, Length: 131, dtype: object

In [499]:
#y = y.to_numpy()
y = y[0]
y.shape = (4000,1)
y

array([[ 0.        ],
       [ 0.01050175],
       [-0.00580046],
       ...,
       [ 0.00134409],
       [ 0.        ],
       [ 0.00813008]])

In [500]:
y_d = y.copy()
y_d[np.where(y_d > 0)] = 1
y_d[np.where(y_d < 0)] = 0
np.unique(y_d, return_counts=True)

(array([0., 1.]), array([2270, 1730]))

In [501]:
X.shape

(4000, 116, 60)

In [502]:
X = X[:,:,:10]
X.shape

(4000, 116, 10)

In [503]:
X = np.transpose(X, axes=(0,2,1))
X.shape

(4000, 10, 116)

In [504]:
model = keras.models.load_model('./data/Models/1min95acc_030920')

In [505]:
preds = model.predict(X)

In [506]:
preds

array([[9.9999988e-01, 1.6716103e-07],
       [1.8119322e-07, 9.9999976e-01],
       [9.9998832e-01, 1.1724805e-05],
       ...,
       [6.0427058e-03, 9.9395728e-01],
       [9.9999845e-01, 1.5581271e-06],
       [3.6261679e-07, 9.9999964e-01]], dtype=float32)

In [507]:
np.argmax(preds, axis=1)

array([0, 1, 0, ..., 1, 0, 1])

In [508]:
y_preds = np.argmax(preds, axis=1)

In [509]:
y_true = y_d.reshape(4000)

In [510]:
y_d

array([[0.],
       [1.],
       [0.],
       ...,
       [1.],
       [0.],
       [1.]])

In [511]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_preds)

0.945

In [157]:
X_old = np.load('./data/prepared/august25screenfixed/numpy_matrices/X.npy')
X_old.shape

(30, 4000, 116, 60)

In [158]:
X_old[0].shape

(4000, 116, 60)

In [159]:
X_old = X_old[0]

In [160]:
X_old.shape, X.shape

((4000, 116, 60), (4000, 10, 116))

In [162]:
#X_old = X_old[:,:,:10]
X_old = np.transpose(X_old, axes=(0,2,1))
X_old.shape

(4000, 10, 116)

In [354]:
X_old == X

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr

In [165]:
y_old = np.load('./data/prepared/august25screenfixed/numpy_matrices/y_br.npy')
y_old[0].shape

(4000,)

In [166]:
y_old = y_old[0]

In [187]:
y_oldd = y_old.copy()
y_oldd[np.where(y_oldd > 0)] = 1
y_oldd[np.where(y_oldd < 0)] = 0
np.unique(y_oldd, return_counts=True)

(array([0., 1.]), array([2217, 1783]))

In [188]:
y_oldtrue = y_oldd.reshape(4000)

In [189]:
accuracy_score(y_oldtrue, y_preds)

0.9665

In [190]:
y_oldtrue == y_true

array([False, False,  True, ...,  True, False, False])

In [355]:
y == y_old

array([[False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [356]:
y[:10]

array([[-0.00141743],
       [-0.00070822],
       [-0.00035398],
       [-0.00035386],
       [ 0.00070822],
       [ 0.00212917],
       [-0.00141743],
       [-0.00070822],
       [ 0.        ],
       [-0.00423131]])

In [357]:
y_old[:10]

array([ 0.00070822,  0.00212917, -0.00141743, -0.00070822,  0.        ,
       -0.00423131,  0.00070572,  0.        ,  0.        ,  0.00247612])

In [358]:
y.shape, y_old.shape

((4000, 1), (4000,))

In [198]:
y.shape = (4000)

In [199]:
y == y_old

array([False, False, False, ..., False, False, False])

In [201]:
y_old_and_new = np.load('./data/prepared/august25screenfixed/numpy_matrices/y_br2.npy')

In [204]:
y_old_and_new[0] == y_old

array([ True,  True,  True, ...,  True,  True,  True])

In [224]:
y_old_and_new[0][:10]

array([ 0.00070822,  0.00212917, -0.00141743, -0.00070822,  0.        ,
       -0.00423131,  0.00070572,  0.        ,  0.        ,  0.00247612])

In [222]:
yfl = np.zeros([1,4000])

df = load_set(stocks[0], data_dir, suffix)
    
df = df.dropna(axis=0)

e = df.shape[0]
l = e - 4000 - 59

df['y'] = df['%close'].shift(1)
y_fl = df.iloc[l:e-59]['y']
yfl[0] = y_fl.to_numpy() 

In [223]:
yfl[0]

array([ 0.00070822,  0.00212917, -0.00141743, ..., -0.00142653,
        0.00142857, -0.00190114])

In [245]:
y_fl[::-1]

84     -0.001901
85      0.001429
86     -0.001427
87     -0.000476
88      0.002382
          ...   
4080    0.000000
4081   -0.000708
4082   -0.001417
4083    0.002129
4084    0.000708
Name: y, Length: 4000, dtype: float64

In [359]:
y_

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

#### 2) Using validation data, model returns by backtesting the system

#### 3) Modify the backtesting system to include some penalty for trading (transaction costs)

### II) Modify the model to protect against minimal movement environments (w/ bid/ask spread)